<a href="https://colab.research.google.com/github/Janengh69/ipynb/blob/master/mlp_KDD_using_tenssorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Dir of total dataset
total_dataset_dir = "drive/My Drive/kdd99/kddcup.data_10_percent_corrected"
training_set_dir = "drive/My Drive/kdd99/train.csv"
test_set_dir = "drive/My Drive/kdd99/test.csv"
debug_set_dir = "debug.csv"
neural_network_model_file = "drive/My Drive/kdd99/model.ckpt"

#Parse total dataset

col_names = ["duration","protocol_type","service","flag","src_bytes",
"dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
"logged_in","num_compromised","root_shell","su_attempted","num_root",
"num_file_creations","num_shells","num_access_files","num_outbound_cmds",
"is_host_login","is_guest_login","count","srv_count","serror_rate",
"srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
"diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
"dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
"dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

num_features = [
    "duration","protocol_type","service","flag", "src_bytes",
    "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins",
    "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root",
    "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds",
    "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate",
    "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate",
    "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate",
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate","label"
]


In [21]:
# Training set

training_set = pandas.read_csv(training_set_dir, names=num_features)
training_label = training_set["label"]
print(training_label)
training_label = training_label.as_matrix()
training_set.drop(['label'], axis=1, inplace=True)


#Test set

test_set = pandas.read_csv(test_set_dir, names = num_features)
test_label = test_set["label"]
test_label = test_label.as_matrix()

test_set.drop(['label'], axis=1, inplace=True)

#Debug set

#debug_set = pandas.read_csv(debug_set_dir,  names = num_features)
#debug_labels = debug_set["label"]
#debug_labels = debug_labels.as_matrix()
#debug_set.drop(['label'], axis=1, inplace=True)



0.0    18
0.0    18
0.0    18
0.0     9
0.0    11
       ..
0.0    18
0.0    18
0.0    18
0.0    18
0.0     9
Name: label, Length: 345814, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [40]:

# Setup input and output
input_set = training_set.as_matrix()
print(input_set)
# Add ones for bias and x inputs
N, M = input_set.shape
input_x = np.ones((N, M + 1))
input_x[:, :-1] = input_set


#Sizes
epoch_num = 50
dataset_size = training_set["duration"].count()
input_size = len(num_features)
output_size = 23
hidden_size = 500
batch_size = 200000


# Setup y outputs
output_y = np.zeros((dataset_size, output_size), dtype=float)
for j in range(dataset_size):
    output_y[j, training_label[j]] = 1.0;



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


[[0.         0.21538462 0.9        ... 0.         0.         0.81818182]
 [0.         0.21538462 0.9        ... 0.         0.         0.81818182]
 [0.         0.21538462 0.9        ... 0.         0.         0.81818182]
 ...
 [0.         0.21538462 0.9        ... 0.         0.         0.81818182]
 [0.         0.21538462 0.9        ... 0.         0.         0.81818182]
 [0.5        0.69230769 0.5        ... 0.         0.         0.40909091]]


In [0]:
with tf.device("/gpu:0"):
    #Place holder
    tf.reset_default_graph()
    x = tf.placeholder(shape=(None , input_size), dtype=tf.float64, name='X')
    y = tf.placeholder(shape=(None , output_size ), dtype=tf.float64, name='y')

    # Randomize Weights
    W1 = tf.Variable(tf.truncated_normal([input_size, hidden_size ], stddev=0.01, dtype=tf.float64), dtype=tf.float64)
    W2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size ], stddev=0.01, dtype=tf.float64), dtype=tf.float64)
    W3 = tf.Variable(tf.truncated_normal([hidden_size, output_size ], stddev=0.01, dtype=tf.float64), dtype=tf.float64)


    # Forward propagation
    A1 =  tf.nn.sigmoid (tf.matmul(x, W1))
    A2 =  tf.nn.sigmoid  (tf.matmul(A1, W2))
    yhat = (tf.matmul(A2, W3))

      #Loss function
      #loss = tf.reduce_sum(tf.square(yhat - y))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = yhat, labels =y))


    #Train
    train_op = tf.train.AdamOptimizer(learning_rate=0.007).minimize(loss)
    tf.set_random_seed(1234)

In [46]:
#sess = tf.Session()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    loss_plot = []
    # for i in range(epoch_num):
    #     print('EPOCH', i)

    #     #Batch
    #     for k in range(round(int(dataset_size)/batch_size)):
    #         start = k*batch_size
    #         end = start + batch_size
    #         _, loss_val = sess.run([train_op, loss] , feed_dict={x: input_x[start:end,:], y: output_y[start:end,:]})

    #     loss_plot.append(loss_val)
    #     print("Loss: ", loss_val)

    # saver = tf.train.Saver()
    # # # Save data
    # saver.save(sess, neural_network_model_file)
    # #Plot
    # plt.plot(loss_plot)
    # plt.ylabel('Loss')
    # plt.show()

    # # Load data
    sess = tf.Session()
    saver = tf.train.import_meta_graph(neural_network_model_file+".meta")
    saver.restore(sess, 'drive/My Drive/kdd99/model.ckpt')


    #Test data

    # Setup input and output
    test_input_set = test_set.as_matrix()

    # Add ones for bias and x inputs
    N, M = test_input_set.shape
    test_input_x = np.ones((N, M + 1))
    test_input_x[:, :-1] = test_input_set

    test_dataset_size = test_set["duration"].count()

    # Setup y outputs
    test_output_y = np.zeros((test_dataset_size, output_size), dtype=float)
    for j in range(test_dataset_size):
        # print(test_output_y[j])
        # print(test_label[j])
        test_output_y[j, test_label[j]] = 1.0;

    total_tests = 0
    correct = 0
    confusion = np.zeros((22, 22))
    for k in range(round(int(test_dataset_size) / batch_size)):
        start = k * batch_size
        end = start + batch_size
        test_ = test_output_y[start:end, :]
        y_est_np = sess.run(yhat, feed_dict={x: test_input_x[start:end,:] , y: test_ })
        batch_correct = [estimate.argmax(axis=0) == target.argmax(axis=0)
                   for estimate, target in zip(y_est_np, test_ )]
        total_tests = total_tests + len(batch_correct)
        correct = correct + sum(batch_correct)
        # print(correct)
        # print(total_tests)
        confusion = confusion + confusion_matrix(test_label,  np.argmax(y_est_np, axis=1)   )

    
    # print(test_input_x)
    # print(test_output_y)
    y_est_np = sess.run(yhat, feed_dict={x: test_input_x, y: test_output_y })

    correct = [estimate.argmax(axis=0) == target.argmax(axis=0)
                for estimate, target in zip(y_est_np, test_output_y )]

    accuracy = 100 * sum(correct) / len(correct)
    confusion = confusion_matrix(test_label,  np.argmax(y_est_np, axis=1)   )
    #f1 = f1_score(test_label, np.argmax(y_est_np, axis=1))
    stats = precision_recall_fscore_support(test_label , np.argmax(y_est_np, axis=1) , average='weighted')

    # Calculate the prediction accuracy
    #accuracy = 100*(correct/total_tests)
    print('Network  accuracy: %.2f%%' % accuracy)
    #print('Number of tests: %d' % total_tests)
    print(stats)

    df = pandas.DataFrame(confusion)
    # print(df)
    df.to_csv("confusion.csv" )

    # df = pandas.DataFrame(f1)
    # df.to_csv("f1_score.csv")

    sess.close()


INFO:tensorflow:Restoring parameters from drive/My Drive/kdd99/model.ckpt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Network  accuracy: 98.16%
(0.9645493404674118, 0.9815798174175309, 0.9727698200029132, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
# print(output_y[1])
import json
out_list = list()
test_list = list()
with open('drive/My Drive/kdd99/data.json', 'r') as file:
  data = json.load(file)
  for ind in np.argmax(y_est_np, axis=1):
    out_list.append([number for number, name in data.items() if name==str(ind)])


In [0]:
error_list =list()
with open('drive/My Drive/kdd99/error.txt', 'w+') as file:
  for i in range(len(test_)):
    if out_list[i] != [number for number, name in data.items() if name==str(test_label[i])]:
      # print(out_list[i], [number for number, name in data.items() if name==str(test_label[i])])
      file.write("".join(out_list[i]) + "  " + "".join([number for number, name in data.items() if name==str(test_label[i])]) + '\n') 